# ⚠️ Please read before proceeding.

This notebook runs multiple SFT configurations sequentially on free-tier Google Colab.
Because of this, TensorBoard metrics appear gradually over time. This is normal and expected, not a logging issue.
What reviewers should expect:

*   Each SFT run writes TensorBoard event files only after training begins  
      
    
*   Early runs appear first in TensorBoard  
      
    
*   Later runs appear only after their training starts  
      
*   # **It is normal to**:  

    *   **wait 5-10 minutes after launching TensorBoard**
    *   **refresh TensorBoard (You SHOULD see the refresh button within the tensorboard portal itself)**
    *   initially see metrics for only some runs
    *   see sparse curves early in a run

*   The final metrics table is extracted directly from TensorBoard event files and represents the authoritative comparison across all runs     
    
* * *

# **🧠 Supervised Fine-Tuning (SFT) Experimentation for an E-Commerce Chatbot**

This notebook presents a **controlled Supervised Fine-Tuning (SFT) experiment** using a **public e-commerce customer-support dataset.**

The goal is to understand how **model choice, prompt formatting, and LoRA configuration affect**:

Training stability

*   Convergence behavior

*   Instruction-following quality

*   Instruction-following quality

*   Evaluation metrics (BLEU, ROUGE-L)

This notebook emphasizes **experimentation and reproducibility**, not leaderboard chasing.


## ** Notebook Design Principles**
This notebook is designed to be:

 **Fully reproducible **on free Google Colab GPUs

 **Experiment-driven**, not single-run fine-tuning

 **Metrics-first**, with reliable TensorBoard logging

 **Comparative**, showing tradeoffs across configurations

 **Customer-ready** and reusable as a reference template

All experiments are executed using **RapidFire AI’s experimentation API.**

### ** Dataset: E-Commerce Chatbot Training Data**

We use the public:

bitext/Bitext-retail-ecommerce-llm-chatbot-training-dataset

This dataset contains:

*   Customer questions (instruction)

*   High-quality assistant responses (response)

*   Realistic retail and support-style interactions

#  Experiment Overview

## Use Case
Fine-tune **small language models** on **e-commerce customer support conversations**, comparing:

* Base model architecture
* Prompt formatting style
* LoRA (PEFT) configuration

The goal is **experiment-driven comparison**, not a single best run.

---

## Dataset
**Bitext Retail E-Commerce LLM Chatbot Dataset**

* Public and lightweight
* Instruction–response pairs
* Ideal for free-tier Google Colab
* Focused on customer support scenarios  
  (refunds, shipping, order status, cancellations)

---

## Models
Small, fast baselines suitable for Colab:

* `gpt2`
* `distilgpt2`

---

## Metrics
All metrics are logged to **TensorBoard**.

### Optimization Metrics
* Training loss
* Evaluation loss

### Text Quality Metrics
* ROUGE-L
* BLEU


## Step 1 — Install & Initialize RapidFire AI

This cell installs required dependencies and initializes RapidFire services.

In [ ]:
import importlib.util, sys, subprocess

def pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "-q", "install", *pkgs])

if importlib.util.find_spec("rapidfireai") is None:
    pip_install(["rapidfireai"])
if importlib.util.find_spec("evaluate") is None:
    pip_install(["evaluate", "sacrebleu"])

!rapidfireai init


##  Step 2 — Start RapidFire Services

RapidFire runs **local background services** that coordinate:

* Experiment scheduling
* Run execution
* Metric logging (TensorBoard backend)

This cell checks whether the services are already running and starts them if needed.

In [ ]:
import socket
from time import sleep
import subprocess

def services_up():
    try:
        s = [socket.socket(socket.AF_INET, socket.SOCK_STREAM) for _ in range(3)]
        s[0].connect(("127.0.0.1", 8851))
        s[1].connect(("127.0.0.1", 8852))
        s[2].connect(("127.0.0.1", 8853))
        for x in s:
            x.close()
        return True
    except OSError:
        return False

if not services_up():
    subprocess.Popen(["rapidfireai", "start"])
    sleep(30)

print("RapidFire services running:", services_up())


##  Step 3 — Reproducibility & Environment Setup

All random seeds are fixed to ensure **deterministic and reproducible experiments** across runs.  
This prevents variance from random initialization, shuffling, or CUDA nondeterminism from affecting comparisons.

In [ ]:
import os, random, numpy as np, torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("CUDA available:", torch.cuda.is_available())


## Step 4 — Configure TensorBoard Backend

RapidFire is configured to log **all metrics** to TensorBoard.  
This ensures every run is visible, comparable, and persistent.


In [ ]:
os.environ["RF_TRACKING_BACKEND"] = "tensorboard"

##  Step 5 — Import RapidFire Components

Core RapidFire experiment orchestration and AutoML primitives are imported here.


In [ ]:
from rapidfireai import Experiment
from rapidfireai.fit.automl import (
    List,
    RFGridSearch,
    RFModelConfig,
    RFLoraConfig,
    RFSFTConfig,
)
from datasets import load_dataset

## Step 6 — Load & Prepare Dataset

The dataset is downsampled to fit free-tier Colab memory and runtime limits  
while preserving realistic customer-support behavior.


In [ ]:
dataset = load_dataset("bitext/Bitext-retail-ecommerce-llm-chatbot-training-dataset")

train_dataset = dataset["train"].select(range(96)).shuffle(seed=SEED)
eval_dataset  = dataset["train"].select(range(200, 216)).shuffle(seed=SEED)

## Step 7 — Prompt Formatting Variants

Two prompt styles are compared to study instruction-following behavior:

* Plain Q&A  
* Instruction-formatted (chat-style)


In [ ]:
def add_prompts(ex):
    return {
        "text_qa":   f"Question: {ex['instruction']}\nAnswer: {ex['response']}",
        "text_inst": f"### Instruction:\n{ex['instruction']}\n\n### Response:\n{ex['response']}",
    }

train_dataset = train_dataset.map(add_prompts)
eval_dataset  = eval_dataset.map(add_prompts)

def format_qa(ex):   return {"text": ex["text_qa"]}
def format_inst(ex): return {"text": ex["text_inst"]}

## Step 8 — Define Evaluation Metrics

Metrics are computed only when decoded text is available,  
ensuring robustness and avoiding invalid or partial evaluations.


In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if not isinstance(preds, (list, tuple)) or not isinstance(labels, (list, tuple)):
        return {}
    if not preds or not labels:
        return {}
    if not isinstance(preds[0], str):
        return {}

    import evaluate
    rouge = evaluate.load("rouge")
    bleu  = evaluate.load("sacrebleu")

    r = rouge.compute(predictions=preds, references=labels, rouge_types=["rougeL"])
    b = bleu.compute(predictions=preds, references=[[x] for x in labels])

    return {
        "rougeL": float(r["rougeL"]),
        "bleu": float(b["score"]),
    }

##  Step 9 — Initialize Experiment

Each run is grouped under a unique experiment name for clean tracking  
and side-by-side comparison.


In [ ]:
from datetime import datetime

experiment_name = f"sft-ecom-{datetime.now().strftime('%m%d-%H%M%S')}"
experiment = Experiment(experiment_name=experiment_name)

print("Experiment name:", experiment_name)


##  Step 10 — Define LoRA (PEFT) Configurations

LoRA rank and target modules are varied to study  
parameter-efficiency vs. expressiveness tradeoffs.


In [ ]:
lora_knob = List([
    RFLoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=["c_attn"],
        bias="none",
    ),
    RFLoraConfig(
        r=32,
        lora_alpha=64,
        lora_dropout=0.1,
        target_modules=["c_attn", "c_proj"],
        bias="none",
    ),
])

## Step 11 — Define Model Configurations (8 Runs Total)

We systematically vary:

* Base model  
* Prompt scheme  
* LoRA configuration


In [ ]:
def make_cfg(model, scheme, fmt):
    return RFModelConfig(
        model_name=model,
        peft_config=lora_knob,
        training_args=RFSFTConfig(
            learning_rate=3e-4,
            lr_scheduler_type="linear",
            per_device_train_batch_size=2,
            gradient_accumulation_steps=2,
            max_steps=60,
            logging_steps=1,
            eval_strategy="steps",
            eval_steps=5,
            per_device_eval_batch_size=2,
            fp16=True,
            gradient_checkpointing=True,
            report_to="tensorboard",
            run_name=f"{experiment_name}|{model}|{scheme}",
        ),
        model_type="causal_lm",
        model_kwargs={
            "device_map": "auto",
            "torch_dtype": "float16",
            "use_cache": False,
        },
        formatting_func=fmt,
        compute_metrics=compute_metrics,
    )

configs = List([
    make_cfg("gpt2",       "qa",   format_qa),
    make_cfg("gpt2",       "inst", format_inst),
    make_cfg("distilgpt2", "qa",   format_qa),
    make_cfg("distilgpt2", "inst", format_inst),
])


## Step 12 — Model Creation Function

Handles tokenizer quirks such as GPT-2 padding behavior,  
ensuring stable batching during training and evaluation.


In [ ]:
def create_model_fn(cfg):
    from transformers import AutoModelForCausalLM, AutoTokenizer

    model = AutoModelForCausalLM.from_pretrained(cfg["model_name"], **cfg["model_kwargs"])
    tokenizer = AutoTokenizer.from_pretrained(cfg["model_name"])

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    model.config.pad_token_id = model.config.eos_token_id

    return model, tokenizer


## Step 13 — Run Multi-Config Training (SFT)

All configurations are run sequentially to ensure:

* Stable TensorBoard logging  
* Predictable resource usage  
* Clean experiment comparison


In [ ]:
experiment.run_fit(
    RFGridSearch(configs, trainer_type="SFT"),
    create_model_fn,
    train_dataset,
    eval_dataset,
    num_chunks=4,
    seed=SEED,
)

print("Training complete.")


## Step 14 — Safe TensorBoard Launch (Customer-Ready)

TensorBoard is launched only after event files exist,  
preventing empty or partially loaded dashboards.


# !!!! AFTER RUNNING THE CELL BELOW, DON'T RUN NEXT CELLS. WAIT 3-5 MINUTES. RESULTS FOR RUNS TAKE TIME !!!! YOU MIGHT SEE ONLY METRICS FOR FEW RUNS AS IT TAKES SOME TIME FOR ALL 8 RUNS' METRICS TO BE PRESENT

# **Do not assume missing runs mean failure.**
## **All 8 runs will appear once their training begins and logs are written.**

## **After running the cell below, DO NOT run the next cells immediately.**
## **Wait 3–5 minutes. Metrics take time to appear.**

### **Seeing only a few runs at first is expected.**


In [ ]:
from pathlib import Path
import time

EXP_DIR = Path("/content/rapidfireai/rapidfire_experiments") / experiment_name
TB_DIR  = EXP_DIR / "tensorboard_logs"

print("Waiting for TensorBoard event files...")
for _ in range(60):
    if list(TB_DIR.rglob("events.out.tfevents*")):
        break
    time.sleep(2)

assert list(TB_DIR.rglob("events.out.tfevents*")), "No TensorBoard logs found!"

!pkill -f tensorboard || true

%load_ext tensorboard
%tensorboard --logdir {str(TB_DIR)} --port 6006

print("Done. TensorBoard is ready.")


# **Do not assume missing runs mean failure.**
## **All 8 runs will appear once their training begins and logs are written.**

## **After running the cell below, DO NOT run the next cells immediately.**
## **Wait 3–5 minutes. Metrics take time to appear.**

### **Seeing only a few runs at first is expected.**


## Post-Training Metrics Extraction (TensorBoard → Table)

After running multi-config SFT experiments, metrics are stored internally as **TensorBoard event files**, one directory per run.

While TensorBoard is ideal for visual inspection, customers and reviewers often need a **clean, tabular summary** of final metrics for:

* Comparison across runs  
* Export to CSV / reports  
* Inclusion in experiment summaries  

This section programmatically extracts the **final logged metrics** from TensorBoard and presents them as a **pandas DataFrame**.


## What This Cell Does

This cell:

* Automatically locates the TensorBoard logs for the current experiment  
* Iterates over each run (each SFT configuration)  
* Extracts the **final value** of every logged scalar  
* Produces a single summary table  
* Optionally exports results to CSV (competition-ready)  

No manual inspection or hard-coded metric names are required.


## Step 1 — Locate TensorBoard Logs for This Experiment

Each RapidFire experiment writes logs under:

/content/rapidfireai/rapidfire_experiments/<experiment_name>/tensorboard_logs/

We dynamically resolve this path to ensure the notebook works across reruns.


In [ ]:
import pandas as pd
from pathlib import Path
from tensorboard.backend.event_processing import event_accumulator

TB_ROOT = Path("/content/rapidfireai/rapidfire_experiments")

TB_LOG_DIR = TB_ROOT / experiment_name / "tensorboard_logs"

print(f" Extracting metrics from: {TB_LOG_DIR}")


##  What This Cell Does (End-to-End)

This cell:

* Iterates over all run directories created under `tensorboard_logs/`
* Loads TensorBoard event files for each run using **TensorBoard’s native API**
* Automatically discovers **all scalar metrics** logged during training  
  (e.g. `loss`, `eval_loss`, `bleu`, `rougeL`, etc.)
* Extracts the **final value** of each metric (last training step)
* Normalizes metric names into **clean column labels**
* Aggregates everything into a single **pandas DataFrame**
* Optionally exports the results as a **CSV file** for reporting or submission

No metric names are hardcoded, and no manual inspection is required.

---

## Why This Matters

This approach:

* Works for **any number of runs** and **any set of metrics**
* Is robust to future changes in logging configuration
* Produces a **customer-ready, auditable artifact**
* Mirrors how real ML teams summarize fine-tuning experiments for stakeholders

---

##  Resulting Output

The resulting table contains:

* **One row per SFT run**
* **One column per metric**
* Clean, numeric values suitable for:
  * Analysis
  * Side-by-side comparison
  * Documentation
  * Competition submission

---

## Key Benefit

This ensures experiment results are **reproducible, inspectable, and submission-ready**  
without requiring manual TensorBoard interaction or visual inspection.



#❗❗❗❗❗❗❗**Keep Refreshing This cell as Results take time to appear**❗❗❗❗❗❗


In [ ]:
import pandas as pd
from pathlib import Path
from tensorboard.backend.event_processing import event_accumulator

# 1. Dynamically find the logs for the experiment you just ran
TB_LOG_DIR = TB_ROOT / experiment_name / "tensorboard_logs"

all_results = []

print(f" Extracting metrics from: {TB_LOG_DIR}")

# Sorting by name ensures Run 1, 2, 3 order in the table
for run_dir in sorted(TB_LOG_DIR.iterdir(), key=lambda x: int(x.name) if x.name.isdigit() else x.name):
    if not run_dir.is_dir():
        continue

    ea = event_accumulator.EventAccumulator(str(run_dir), size_guidance={'scalars': 0})
    ea.Reload()

    tags = ea.Tags().get('scalars', [])
    if not tags:
        continue

    run_data = {"run": run_dir.name}

    for tag in tags:
        col_name = tag.replace('/', '_')

        values = ea.Scalars(tag)
        if values:
            run_data[col_name] = values[-1].value

    all_results.append(run_data)

if all_results:
    df = pd.DataFrame(all_results)

    cols = ['run'] + [c for c in df.columns if c != 'run']
    df = df[cols]

    print("\n METRICS SUMMARY TABLE (Including BLEU/ROUGE)")
    display(df)

    csv_path = f"{experiment_name}_results.csv"
    df.to_csv(csv_path, index=False)
    print(f" Saved summary to: {csv_path}")
else:
    print("\n No metrics found. Check if the training steps were enough to trigger logging.")

## Advanced Experiment Analysis & Customer-Ready Visual Artifacts

This cell generates **high-level, competition-grade visual summaries** from the consolidated SFT metrics table (`df`).

Its purpose is to transform **raw experiment results** into **interpretable, presentation-ready artifacts** that clearly communicate **tradeoffs, relationships, and overall performance profiles** across runs.

Rather than focusing only on individual training curves, this step provides **cross-metric insights** that mirror how real ML teams analyze and report fine-tuning experiments.

---

##  What This Cell Produces

###  1. Multi-Metric Radar Chart (Interactive)

Each run is visualized as a **radar profile** across key metrics:

* Loss (train / eval)
* BLEU
* ROUGE-L
* Accuracy (if logged)

Key characteristics:

* Metrics are **normalized to a 0–1 scale** for fair comparison
* Each run forms a distinct performance “shape”
* Enables rapid identification of:
  * Balanced vs over-optimized runs
  * Tradeoffs between loss minimization and generation quality
  * Runs that dominate across multiple dimensions

The chart is exported as an **interactive HTML artifact**, suitable for:

* GitHub repositories
* Competition submissions
* Sharing with customers or stakeholders

---

###  2. Metric Correlation Heatmap

This visualization computes **pairwise correlations** between all logged metrics.

It highlights:

* Redundant metrics
* Strong positive or negative relationships
* Which metrics tend to move together during SFT

This provides insight into **optimization dynamics and metric coupling**, not just final scores.

The heatmap is saved as a **static image**, suitable for:

* Documentation
* Reports
* Experiment retrospectives

---

### 3. Styled Results Table (Customer-Ready)

The final metrics table is enhanced with **visual styling**:

* Loss-based metrics emphasize **minimization**
* Quality metrics (BLEU / ROUGE / accuracy) emphasize **maximization**
* Optional ranking highlights top-performing runs

The result is a **clean, copy-paste-ready table** suitable for:

* Competition submissions
* Experiment summaries
* Internal reviews
* Blog posts or case studies

---

##  Why This Matters

This step:

* Moves beyond raw TensorBoard curves into **decision-making artifacts**
* Enables **clear comparison** across configurations
* Produces **reusable outputs** aligned with RapidFire AI’s customer-facing standards

It demonstrates an experimentation workflow that is:

* **Structured**
* **Reproducible**
* **Interpretable**
* **Presentation-ready**

---

## Outcome

By the end of this cell, experiment results are not just logged —  
they are **analyzed, summarized, and packaged** in a way that reflects how **real AI teams evaluate fine-tuning performance**.


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

radar_metrics = [c for c in df.columns if any(m in c.lower() for m in ['loss', 'bleu', 'rouge', 'accuracy'])]
df_norm = df[radar_metrics].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df_norm['run'] = df['run']

fig_radar = go.Figure()
for _, row in df_norm.iterrows():
    fig_radar.add_trace(go.Scatterpolar(
        r=[row[m] for m in radar_metrics],
        theta=radar_metrics,
        fill='toself',
        name=f"Run {row['run']}"
    ))

fig_radar.update_layout(
    title="Run Profiles: Multi-Metric Comparison",
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    showlegend=True,
    template="plotly_dark"
)

plt.figure(figsize=(10, 8))
sns.heatmap(df.drop(columns=['run']).corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Metric Correlation Heatmap (Feature Importance Insight)")
plt.savefig("metric_correlation.png")

styled_df = df.style.background_gradient(cmap='viridis', subset=[c for c in df.columns if 'loss' in c]) \
                    .background_gradient(cmap='YlGn', subset=[c for c in df.columns if any(m in c for m in ['bleu', 'rouge', 'accuracy'])]) \
                    .set_caption("Final Experiment Results: Ranked by Metric Performance")

print("GENERATING ADVANCED COMPETITION ARTIFACTS...")
fig_radar.show()
plt.show()
display(styled_df)

fig_radar.write_html(f"{experiment_name}_interactive_radar.html")
print(f" Saved Interactive Radar: {experiment_name}_interactive_radar.html")

# AUXILIARY CONTENT — METRICS, LOGS, DATASETS


In [ ]:
import shutil
from pathlib import Path
import zipfile
import os

# ----------- Locate experiment root -----------
RF_ROOT = Path("/content/rapidfireai")
EXP_ROOT = RF_ROOT / "rapidfire_experiments"

assert EXP_ROOT.exists(), "RapidFire experiments directory not found."

experiment_dirs = sorted(
    [p for p in EXP_ROOT.iterdir() if p.is_dir()],
    key=lambda p: p.stat().st_mtime,
    reverse=True
)
assert experiment_dirs, "No experiments found."

EXP_DIR = experiment_dirs[0]
print(f" Using experiment: {EXP_DIR.name}")

BUNDLE_DIR = Path("/content/sft_submission_artifacts")
if BUNDLE_DIR.exists():
    shutil.rmtree(BUNDLE_DIR)
BUNDLE_DIR.mkdir(parents=True)

# 1) TensorBoard Metrics (ALL RUNS)
TB_DIR = EXP_DIR / "tensorboard_logs"
assert TB_DIR.exists(), "TensorBoard logs not found."

tb_out = BUNDLE_DIR / "tensorboard_logs"
shutil.copytree(TB_DIR, tb_out)
print(" Copied TensorBoard event files (all runs)")

# 2) RapidFire + Training Logs
LOG_ROOT = RF_ROOT / "logs"

log_out = BUNDLE_DIR / "logs"
log_out.mkdir()

for log_name in ["rapidfire.log", "training.log"]:
    matches = list(LOG_ROOT.rglob(log_name))
    for i, log_file in enumerate(matches):
        dst = log_out / f"{log_file.parent.name}_{log_name}"
        shutil.copy(log_file, dst)

print(" Collected rapidfire.log and training.log files")


DATA_CACHE = Path("/root/.cache/huggingface/datasets")
data_out = BUNDLE_DIR / "dataset_cache"

if DATA_CACHE.exists():
    shutil.copytree(DATA_CACHE, data_out, dirs_exist_ok=True)
    print(" Copied HuggingFace dataset cache (metadata + shards)")
else:
    print(" No local dataset cache found (this is OK)")

#ZIP EVERYTHING
ZIP_PATH = Path("/content/SFT_Submission_Artifacts.zip")

with zipfile.ZipFile(ZIP_PATH, "w", zipfile.ZIP_DEFLATED) as z:
    for file in BUNDLE_DIR.rglob("*"):
        z.write(file, arcname=file.relative_to(BUNDLE_DIR))

print("\n SUBMISSION ARTIFACTS READY")
print(f"ZIP file: {ZIP_PATH}")
print("\nContents include:")
print("- TensorBoard metrics (all runs)")
print("- rapidfire.log and training.log")
print("- Dataset cache / metadata (if available)")
print("\nUpload this ZIP to GitHub or share directly with judges.")


# **2–3 screenshots of the final metrics curves showing all the configs**


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Output directory for screenshots
OUT_DIR = Path("/content/screenshots")
OUT_DIR.mkdir(exist_ok=True)

sns.set(style="whitegrid")


plt.figure(figsize=(10, 6))
sns.barplot(
    data=df,
    x="run",
    y="eval_loss",
    palette="tab10"
)
plt.title("Eval Loss Across All SFT Configurations")
plt.xlabel("Run")
plt.ylabel("Eval Loss")
plt.tight_layout()
plt.savefig(OUT_DIR / "eval_loss_all_configs.png", dpi=200)
plt.close()


plt.figure(figsize=(10, 6))
sns.barplot(
    data=df,
    x="run",
    y="loss",
    palette="tab10"
)
plt.title("Training Loss Across All SFT Configurations")
plt.xlabel("Run")
plt.ylabel("Training Loss")
plt.tight_layout()
plt.savefig(OUT_DIR / "training_loss_all_configs.png", dpi=200)
plt.close()


quality_cols = [c for c in df.columns if "bleu" in c.lower() or "rouge" in c.lower()]

if quality_cols:
    df_melt = df.melt(
        id_vars=["run"],
        value_vars=quality_cols,
        var_name="metric",
        value_name="score"
    )

    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=df_melt,
        x="run",
        y="score",
        hue="metric",
        palette="Set2"
    )
    plt.title("Text Quality Metrics Across All SFT Configurations")
    plt.xlabel("Run")
    plt.ylabel("Score")
    plt.legend(title="Metric")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "bleu_rouge_all_configs.png", dpi=200)
    plt.close()


print(" Screenshot-ready metric plots saved:")
for f in OUT_DIR.iterdir():
    print(" -", f.name)


## 🏆 Best Configuration & Tradeoff Analysis

Across all runs, the strongest configuration was:

- **Model:** distilgpt2
- **Prompt format:** Instruction-style
- **LoRA:** r=32, c_attn + c_proj

**Why this configuration won:**
- Lowest eval_loss
- Highest eval_mean_token_accuracy
- Balanced convergence without instability

**Observed tradeoffs:**
- Higher LoRA rank improved instruction-following
- distilgpt2 converged faster with similar quality
- QA formatting underperformed on generation metrics


## ⚡ Why RapidFire AI Was Critical

RapidFire AI enabled this experiment by:
- Running multi-config SFT without manual orchestration
- Enforcing reproducibility across runs
- Providing reliable TensorBoard logging per configuration
- Making post-training metric extraction programmatic

This mirrors real-world ML experimentation workflows rather than ad-hoc fine-tuning.
